                                     Reviews-Delhi

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium.common.exceptions import NoSuchElementException
import sys

In [2]:
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}

## Function to get the webpage

In [3]:
def Search(search_query):
    url='https://www.practo.com/search?results_type=doctor&q=%5B%7B%22word%22%3A%22general%20physician%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi&page='+search_query
    time.sleep(1)
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

## Function to scrape the link of individual doctor's profile

In [4]:
link=[]
for i in range(75): #Doctor's profiles are in 75 pages
    search_response=Search(str(i)) 
    soup=BeautifulSoup(search_response.content) #soup has all the content in the page
    links= soup.findAll('a',{'class':"u-color--primary"}) #acessing the tag in which links are present
    for l in links:
        link.append(l.get('href')) #saving all the links of the doctor's profile

In [5]:
df=pd.DataFrame()

df['link']=link #converting links to a dataframe

## Function to change the address of the link(so that we will get the recent reviews)

In [6]:
def clean(row):
    Link=row['link']
    Link=Link.split('?')
    return Link[0]+'/recent'

In [7]:
df['link']=df.apply(lambda row:clean(row),axis=1) #calling function

In [8]:
df=df.drop_duplicates(subset='link').reset_index(drop=True) #dropping all the duplicate links

In [9]:
Links=df['link']

## Initializing selenium 

###### As we have infinite scrolling, selenium will be easy to click the more button

In [10]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path='C:/Users/VIJESH\Desktop/chromedriver.exe')

## Function to click more button (to get all the reviews of the respective doctor)

In [14]:
def review_page(query):    
    driver.get("https://www.practo.com"+Links[query])  #opens the doctor's profile
    for i in range(65):                             #65 times clicks the more reviews button
        try:
            driver.find_element_by_xpath('//button[@data-qa-id="view-more-feedback"]').click() #click by using Xpath
            time.sleep(2)
        except NoSuchElementException:
            pass
    return driver.page_source #returns the page

## Function to scrape the reviews

In [15]:
review=[]
rev=''
recom=[]
import time
for i in range(len(Links)):
    soup = BeautifulSoup(review_page(int(i)), 'lxml')  #soup has the content of the doctor's profile
    for rev_clas in soup.findAll(attrs={'class':"feedback__body"}):#acessing the tag in which reviews are present
        for i in rev_clas.findAll(attrs={'data-qa-id':"review-text"}):#saves all the patient reviews 
            rev=rev+i.text
        review.append(rev)
        rev=''
    for rev_clas in soup.findAll(attrs={'class':"pure-g feedback--item u-cushion--medium-vertical"}):
        for i in rev_clas.findAll(attrs={'class':'u-cushion--small-bottom u-large-font'}):#saves the recommendations
            recom.append(i.text)

In [16]:
len(review)

5203

In [25]:
df_1=pd.DataFrame()

In [26]:
df_1['reviews']=review
df_1['recommendation']=recom

In [27]:
df_1=df_1.drop_duplicates() #droppping all duplicates

## Function to tag the location

In [28]:
def location(row):
    each_row=row['reviews']
    return 'Delhi'

In [29]:
df_1['location']=df_1.apply(lambda row:location(row),axis=1) #calling the function

In [30]:
df_1.head()

,reviews,recommendation,location
0,"Dr Amarpreet treats illness from roots, I got ...",I recommend the doctor,Delhi
1,"I am really thankful to you Sir,for the miracl...",I recommend the doctor,Delhi
2,Very nice experience. *** ********* ** ****** ...,I recommend the doctor,Delhi
3,It was a good experience Doctor is very good...,I recommend the doctor,Delhi
4,We are very happy with the course of action su...,I recommend the doctor,Delhi


In [32]:
df_1.to_csv('doctor_Delhi.csv',index=False) #saving the dataframe as CSV

In [31]:
len(df_1)

4618